<a href="https://colab.research.google.com/github/SanjaySivaraman/DAV/blob/master/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd
import nltk
import plotly.express as px
from google.cloud import bigquery

In [4]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.5MB/s 
     |████████████████████████████████| 552kB 17.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=8d20ca55ca65ff4f165fa37c8fe291f5abcebf9eccba26bb993aeb344e2615b5
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=02a972a24f48e77652fe4af7f0ca2476a3f9377c8dff50f8a77fc8c16ab68087
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [6]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
import re, numpy as np
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [8]:
# Construct a BigQuery client object.
client = bigquery.Client(project='primal-hybrid-282705')

query = """
    SELECT * FROM `gdelt-bq.extra.sourcesbycountry` 
"""
query_job = client.query(query)
master_data = []
for row in query_job:
  each_data = []
  for each_val in row:
    each_data.append(each_val)
  master_data.append(each_data)

In [9]:
df_sources = pd.DataFrame(master_data)
df_sources.columns = ['source','FIPS','CountryName']

In [9]:
df_sources.head(2)

,source,FIPS,CountryName
0,northeastsurfing.com,OC,None
1,alwihdainfo.com,CD,Chad


In [10]:
source_count = df_sources.groupby('CountryName')['source'].nunique().reset_index()

In [11]:
source_count[source_count['CountryName'].isin(['India','Indonesia','Vietnam','Fiji','Papua New Guinea','Samoa','Solomon Islands','East Timor'])]

,CountryName,source
57,East Timor,2
65,Fiji,15
90,India,686
91,Indonesia,97
155,Papua New Guinea,5
172,Samoa,12
183,Solomon Islands,6
219,Vietnam,219


In [12]:
df_source_observatory = df_sources[df_sources['CountryName'].isin(['India','Indonesia','Vietnam','Fiji','Papua New Guinea','Samoa','Solomon Islands','East Timor'])]

In [13]:
df_source_observatory['FIPS'].unique()

array(['FJ', 'IN', 'WS', 'VM', 'ID', 'TT', 'BP', 'PP'], dtype=object)

In [14]:
#df_source_observatory.to_csv('/content/drive/My Drive/World Bank Projects/Covid-19 Analysis/Text Mining/sources_gdelt.csv')

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client(project='primal-hybrid-282705')

query = """
    SELECT * FROM `gdelt-bq.covid19.onlinenewsgeo` WHERE CountryCode='IN' AND DATE(DateTime) >= "2020-01-01" 
"""
query_job = client.query(query)

master_data = []
for row in query_job:
    #Row values can be accessed by field name or index.
    time = row[0]
    link = row[1]
    heading = row[2]
    language = row[4]
    docTone = row[5]
    location = row[7]
    latitude = row[8]
    longitude = row[9]
    Adm1Code = row[11]
    Adm2Code = row[12]
    GeoType = row[13]
    ContextualText = row[14]
    
    master_data.append([time,link,heading,language,docTone,location,latitude,longitude,Adm1Code,Adm2Code,GeoType,ContextualText])

df_master_data = pd.DataFrame(master_data)
df_master_data.columns = ['time','link','heading','language','docTone','location','latitude','longitude','Adm1Code','Adm2Code','GeoType','ContextualText']

In [ ]:
df_master_data['link'].nunique()

In [ ]:
path_save_raw = '/content/drive/My Drive/World Bank Projects/Covid-19 Analysis/Text Mining/Data_Raw/PNG.pkl'
df_master_data.to_pickle(path_save_raw)

In [ ]:
df_master_data = pd.read_pickle('/content/drive/My Drive/World Bank Projects/Covid-19 Analysis/Text Mining/Data_Raw/India.pkl')

In [ ]:
fips_code = '/content/drive/My Drive/World Bank Projects/Covid-19 Analysis/Text Mining/fips_in.xlsx'
df_fips_in = pd.read_excel(fips_code,header=None)
df_fips_in.columns = ['Adm1Code','State Name']

In [ ]:
df_master_data1 = pd.merge(df_master_data,df_fips_in,on='Adm1Code',how='left')

In [ ]:
df_master_data1['Level'] = df_master_data1['Adm1Code'].apply(lambda x: 'Centre' if x=='IN' else 'States')
df_master_data1.groupby('Level')['link'].nunique()

In [ ]:
df_common = df_master_data1.groupby(['link'])['Level'].nunique().reset_index()
len(df_common[df_common['Level']>1])

In [ ]:
df_coverage_state = df_master_data1.groupby(['State Name'])['link'].nunique().reset_index().sort_values(by='link')
df_coverage_state.head(2)

In [ ]:
df_doc_tone = df_master_data1.groupby(['State Name'])['docTone'].mean().reset_index().sort_values(by='docTone')
df_doc_tone.head(40)

In [ ]:
df_cov_tone = pd.merge(df_coverage_state,df_doc_tone,on='State Name')
df_cov_tone['Level'] = df_cov_tone['State Name'].apply(lambda x: 'National' if x=='India - National' else 'States')

In [ ]:
fig = px.scatter(df_cov_tone, x="link", y="docTone", hover_name="State Name",color='Level')
fig.update_layout(
    title="GDELT Total Coverage - India",
    xaxis_title="Number of Articles",
    yaxis_title="Average Document Tone",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [ ]:
def get_context(x):
  if(x!=None):
    text_lower = x.lower()
    if any(ext in text_lower for ext in ['corrupt','corruption','bribe','bribery','fraud','misappropriation','embezzlement']):
      return True
    else:
      return False
  else:
    return False

In [ ]:
df_master_data1['corruption_indicator'] = df_master_data1['heading'].apply(get_context)

In [ ]:
df_with_corruption = df_master_data1[df_master_data1['corruption_indicator']==True]

In [ ]:
df_with_corruption = df_with_corruption.drop_duplicates()

In [ ]:
df_state_corruption_count = df_with_corruption.groupby(['State Name','Level'])['link'].nunique().reset_index().sort_values(by='link')
df_state_corruption_count['State'] = df_state_corruption_count['State Name'].apply(lambda x:x.split(' ')[0])

In [ ]:
fig = px.bar(df_state_corruption_count, x="State", y="link", color='Level')
fig.update_layout(
    title="GDELT Fraud and Corruption - India",
    xaxis_title="State",
    yaxis_title="# of articles on <br> Fraud and Corruption",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="black"
    )
)
fig.show()

In [ ]:
df_with_corruption.head(2)

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df_with_corruption.heading.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
# Sentence Coloring of N Sentences
def topics_per_document(model, corpus, start=0, end=1):
    corpus_sel = corpus[start:end]
    dominant_topics = []
    topic_percentages = []
    for i, corp in enumerate(corpus_sel):
        topic_percs, wordid_topics, wordid_phivalues = model[corp]
        dominant_topic = sorted(topic_percs, key = lambda x: x[1], reverse=True)[0][0]
        dominant_topics.append((i, dominant_topic))
        topic_percentages.append(topic_percs)
    return(dominant_topics, topic_percentages)

dominant_topics, topic_percentages = topics_per_document(model=lda_model, corpus=corpus, end=-1)            

# Distribution of Dominant Topics in Each Document
df = pd.DataFrame(dominant_topics, columns=['Document_Id', 'Dominant_Topic'])
dominant_topic_in_each_doc = df.groupby('Dominant_Topic').size()
df_dominant_topic_in_each_doc = dominant_topic_in_each_doc.to_frame(name='count').reset_index()

# Total Topic Distribution by actual weight
topic_weightage_by_doc = pd.DataFrame([dict(t) for t in topic_percentages])
df_topic_weightage_by_doc = topic_weightage_by_doc.sum().to_frame(name='count').reset_index()

# Top 3 Keywords for each Topic
topic_top3words = [(i, topic) for i, topics in lda_model.show_topics(formatted=False) 
                                 for j, (topic, wt) in enumerate(topics) if j < 3]

df_top3words_stacked = pd.DataFrame(topic_top3words, columns=['topic_id', 'words'])
df_top3words = df_top3words_stacked.groupby('topic_id').agg(', \n'.join)
df_top3words.reset_index(level=0,inplace=True)

from matplotlib.ticker import FuncFormatter

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=120, sharey=True)

# Topic Distribution by Dominant Topics
ax1.bar(x='Dominant_Topic', height='count', data=df_dominant_topic_in_each_doc, width=.5, color='firebrick')
ax1.set_xticks(range(df_dominant_topic_in_each_doc.Dominant_Topic.unique().__len__()))
tick_formatter = FuncFormatter(lambda x, pos: 'Topic ' + str(x)+ '\n' + df_top3words.loc[df_top3words.topic_id==x, 'words'].values[0])
ax1.xaxis.set_major_formatter(tick_formatter)
ax1.set_title('Number of Documents by Dominant Topic', fontdict=dict(size=10))
ax1.set_ylabel('Number of Documents')
ax1.set_ylim(0, 1000)

# Topic Distribution by Topic Weights
ax2.bar(x='index', height='count', data=df_topic_weightage_by_doc, width=.5, color='steelblue')
ax2.set_xticks(range(df_topic_weightage_by_doc.index.unique().__len__()))
ax2.xaxis.set_major_formatter(tick_formatter)
ax2.set_title('Number of Documents by Topic Weightage', fontdict=dict(size=10))

plt.show()

In [ ]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

In [ ]:
df_with_corruption['dominant_topic'] = df_dominant_topic['Dominant_Topic'].values

In [ ]:
df_with_corruption.head(2)

In [ ]:
df_with_corruption.groupby(['State Name','dominant_topic'])['heading'].nunique().reset_index()